![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# 2. Clinical Assertion Status Model v2.7.0

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp version: 2.6.3

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.164.245.220:7077 
 AppName 
 Databricks Shell

# Clinical Assertion Model (with pretrained models)

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = NerDLModel.pretrained("ner_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
# coming from sparknlp_jsl.annotator !!
clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
    
nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][ / ][OK!]

In [0]:
# we also have a LogReg based Assertion Model.
'''
clinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
'''

Out[4]: '\nclinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models") .setInputCols(["sentence", "ner_chunk", "embeddings"]) .setOutputCol("assertion")\n'

In [0]:
import pandas as pd

text = 'Patient has a headache for the last 2 weeks and appears anxious when she walks fast. No alopecia noted. She denies pain'

print (text)

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

,chunks,entities,assertion
0,a headache,PROBLEM,present
1,anxious,PROBLEM,conditional
2,alopecia,PROBLEM,absent
3,pain,PROBLEM,absent


In [0]:
! wget 	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv


In [0]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("pubmed_sample_text_small.csv")\
                
pubMedDF.show(truncate=50)

+--------------------------------------------------+
 text|
+--------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|
BACKGROUND: At present, it is one of the most i...|
OBJECTIVE: To investigate the relationship betw...|
Combined EEG/fMRI recording has been used to lo...|
Kohlschutter syndrome is a rare neurodegenerati...|
Statistical analysis of neuroimages is commonly...|
The synthetic DOX-LNA conjugate was characteriz...|
Our objective was to compare three different me...|
We conducted a phase II study to assess the eff...|
"""Monomeric sarcosine oxidase (MSOX) is a flav...|
We presented the tachinid fly Exorista japonica...|
The literature dealing with the water conductin...|
A novel approach to synthesize chitosan-O-isopr...|
An HPLC-ESI-MS-MS method has been developed for...|
The localizing and lateralizing values of eye a...|
OBJECTIVE: To evaluate the effectiveness and ac...|
For the construction of new combinatorial libra...|
We report the results of a screen for genetic a...|
Intraparenchymal pericatheter cyst is rarely re...|
It is known that patients with Klinefelter's sy...|
+--------------------------------------------------+
only showing top 20 rows

In [0]:
result = model.transform(pubMedDF.limit(100))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| assertion|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
The human KCNJ9 (...|[[document, 0, 95...|[[document, 0, 12...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 17, 19, ...|[[assertion, 17, ...|
BACKGROUND: At pr...|[[document, 0, 14...|[[document, 0, 19...|[[token, 0, 9, BA...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 67, 79, ...|[[assertion, 67, ...|
OBJECTIVE: To inv...|[[document, 0, 15...|[[document, 0, 30...|[[token, 0, 8, OB...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 130, 152...|[[assertion, 130,...|
Combined EEG/fMRI...|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 7, Co...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 9, 26, E...|[[assertion, 9, 2...|
Kohlschutter synd...|[[document, 0, 25...|[[document, 0, 20...|[[token, 0, 11, K...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 20, K...|[[assertion, 0, 2...|
Statistical analy...|[[document, 0, 10...|[[document, 0, 34...|[[token, 0, 10, S...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 34, S...|[[assertion, 0, 3...|
The synthetic DOX...|[[document, 0, 57...|[[document, 0, 10...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 4, 30, s...|[[assertion, 4, 3...|
Our objective was...|[[document, 0, 24...|[[document, 0, 19...|[[token, 0, 2, Ou...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 56, 80, ...|[[assertion, 56, ...|
We conducted a ph...|[[document, 0, 14...|[[document, 0, 24...|[[token, 0, 1, We...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 13, 28, ...|[[assertion, 13, ...|
"""Monomeric sarc...|[[document, 0, 14...|[[document, 0, 14...|[[token, 0, 2, ""...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 3, 36, M...|[[assertion, 3, 3...|
We presented the ...|[[document, 0, 12...|[[document, 0, 26...|[[token, 0, 1, We...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 135, 178...|[[assertion, 135,...|
The literature de...|[[document, 0, 16...|[[document, 0, 13...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 63, 83, ...|[[assertion, 63, ...|
A novel approach ...|[[document, 0, 64...|[[document, 0, 97...|[[token, 0, 0, A,...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 99, 132,...|[[assertion, 99, ...|
An HPLC-ESI-MS-MS...|[[document, 0, 90...|[[document, 0, 24...|[[token, 0, 1, An...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 23, A...|[[assertion, 0, 2...|
The localizing an...|[[document, 0, 72...|[[document, 0, 12...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 42, 98, ...|[[assertion, 42, ...|
OBJECTIVE: To eva...|[[document, 0, 13...|[[document, 0, 43...|[[token, 0, 8, OB...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 62, 80, ...|[[assertion, 62, ...|
For the construct...|[[document, 0, 32...|[[document, 0, 20...|[[token, 0, 2, Fo...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 24, 50, ...|[[assertion, 24, ...|
We report the res...|[[document, 0, 13...|[[document, 0, 38...|[[token, 0, 1, We...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 25, 32, ...|[[assertion, 25, ...|
Intraparenchymal ...|[[document, 0, 10...|[[document, 0, 53...|[[token, 0, 15, I...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 33, I...|[[assertion, 0, 3...|
It is known that ...|[[document, 0, 34...|[[document, 0, 14...|[[token, 0, 1, It...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 31, 52, ...|[[assertion, 31, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
result.select('sentence.result').take(1)

Out[10]: [Row(result=['The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family.', 'Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population.', 'The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively.', 'We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair', '(bp) insertion/deletion.', 'Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle.', 'The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes.'])]

In [0]:
result.select(F.explode(F.arrays_zip('ner_chunk.result',  'ner_chunk.begin', 'ner_chunk.end', 'ner_chunk.metadata', 'assertion.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']['entity']").alias("ner_label"),
        F.expr("cols['3']['sentence']").alias("sent_id"),
        F.expr("cols['4']").alias("assertion"),).show(truncate=False)


+---------------------------------------------+-----+---+---------+-------+---------+
chunk |begin|end|ner_label|sent_id|assertion|
+---------------------------------------------+-----+---+---------+-------+---------+
Kir |17 |19 |TEST |0 |present |
GIRK3 |26 |30 |TEST |0 |present |
chromosome 1q21-23 |188 |205|TREATMENT|1 |present |
a candidate gene forType II diabetes mellitus|210 |254|PROBLEM |1 |possible |
coding exons |357 |368|TEST |2 |absent |
byapproximately |380 |394|TEST |2 |present |
introns |425 |431|TEST |2 |absent |
single nucleotide polymorphisms (SNPs |464 |500|PROBLEM |3 |present |
aVal366Ala substitution |532 |554|TREATMENT|3 |present |
an 8 base-pair |561 |574|PROBLEM |3 |present |
bp) |577 |579|TEST |4 |present |
Ourexpression studies |601 |621|TEST |5 |present |
The characterization of the KCNJ9 gene |773 |810|TEST |6 |present |
furtherstudies |830 |843|TREATMENT|6 |present |
KCNJ9 protein |868 |880|TEST |6 |present |
evaluation |892 |901|TEST |6 |present |
Type II diabetes |940 |955|PROBLEM |6 |present |
the treatment |67 |79 |TREATMENT|0 |present |
breast cancer |84 |96 |PROBLEM |0 |present |
anthracyclines |167 |180|TREATMENT|0 |present |
+---------------------------------------------+-----+---+---------+-------+---------+
only showing top 20 rows

## Writing a generic Assertion + NER function

In [0]:
from pyspark.sql.functions import monotonically_increasing_id


def get_base_pipeline (embeddings = 'embeddings_clinical'):

  documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line

  sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP

  tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
  word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  model = nlpPipeline.fit(empty_data)

  return model



def get_clinical_assertion (embeddings, spark_df, nrows = 100, model_name = 'ner_clinical'):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
  loaded_ner_model = NerDLModel.pretrained(model_name, "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

  ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

  # Assertion model trained on i2b2 (sampled from MIMIC) dataset
  # coming from sparknlp_jsl.annotator !!
  clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
      .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")
      

  base_model = get_base_pipeline (embeddings)

  nlpPipeline = Pipeline(stages=[
      base_model,
      loaded_ner_model,
      ner_converter])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  model = nlpPipeline.fit(empty_data)

  result = model.transform(spark_df.limit(nrows))

  result = result.withColumn("id", monotonically_increasing_id())

  result_df = result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata', 'assertion.result')).alias("cols")) \
  .select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label"),
        F.expr("cols['2']").alias("assertion"))\
        .filter("ner_label!='O'")

  return result_df

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_clinical'

nrows = 1000

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_clinical download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+--------------------+---------+---------+
 chunk|ner_label|assertion|
+--------------------+---------+---------+
 Kir| TEST| present|
 GIRK3| TEST| present|
 chromosome 1q21-23|TREATMENT| present|
a candidate gene ...| PROBLEM| possible|
 coding exons| TEST| absent|
 byapproximately| TEST| present|
 introns| TEST| absent|
single nucleotide...| PROBLEM| present|
aVal366Ala substi...|TREATMENT| present|
 an 8 base-pair| PROBLEM| present|
 bp)| TEST| present|
Ourexpression stu...| TEST| present|
The characterizat...| TEST| present|
 furtherstudies|TREATMENT| present|
 KCNJ9 protein| TEST| present|
 evaluation| TEST| present|
 Type II diabetes| PROBLEM| present|
 the treatment|TREATMENT| present|
 breast cancer| PROBLEM| present|
 anthracyclines|TREATMENT| present|
+--------------------+---------+---------+
only showing top 20 rows

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_posology download started this may take some time.
Approximate size to download 13.7 MB
[ | ][ / ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

entry_data = spark.createDataFrame([["The patient didn't take a capsule of Advil"]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, model_name)

ner_df.show()

ner_posology download started this may take some time.
Approximate size to download 13.7 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+-------+---------+---------+
 chunk|ner_label|assertion|
+-------+---------+---------+
capsule| FORM| present|
 Advil| DRUG| null|
+-------+---------+---------+

In [0]:
import pandas as pd

def get_clinical_assertion_light (light_model, text):

  light_result = light_model.fullAnnotate(text)[0]

  chunks=[]
  entities=[]
  status=[]

  for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
      
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      status.append(m.result)
          
  df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

  return df

In [0]:
clinical_text = """
Patient with severe fever and sore throat. 
He shows no stomach pain and he maintained on an epidural and PCA for pain control.
He also became short of breath with climbing a flight of stairs.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer.
"""

light_model = LightPipeline(model)

get_clinical_assertion_light (light_model, clinical_text)


,chunks,entities,assertion
0,severe fever,PROBLEM,present
1,sore throat,PROBLEM,present
2,stomach pain,PROBLEM,absent
3,an epidural,TREATMENT,present
4,PCA,TREATMENT,present
5,pain control,PROBLEM,present
6,short of breath,PROBLEM,conditional
7,CT,TEST,present
8,lung tumor,PROBLEM,present
9,Alzheimer,PROBLEM,associated_with_someone_else


## Assertion with BioNLP (Cancer Genetics) NER

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_bionlp'

nrows = 100

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show(truncate = False)

ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

# Train a custom Assertion Model

In [0]:
!wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/ner/i2b2_assertion_sample.csv

In [0]:
%sh ls -la

total 28944
drwxr-xr-x 1 root root 4096 Nov 1 13:53 .
drwxr-xr-x 1 root root 4096 Nov 1 11:54 ..
-rw-r--r-- 1 root root 514752 Nov 1 12:56 amazon_alexa.tsv
-rw-r--r-- 1 root root 514752 Nov 1 13:16 amazon_alexa.tsv.1
drwxr-xr-x 2 root root 4096 Jan 1 1970 conf
-rw-r--r-- 1 root root 717 Nov 1 11:54 derby.log
drwxr-xr-x 3 root root 4096 Nov 1 11:54 eventlogs
drwxr-xr-x 3 root root 4096 Nov 1 12:03 file:
drwxr-xr-x 2 root root 4096 Nov 1 14:00 ganglia
drwxr-xr-x 2 root root 4096 Nov 1 13:11 gc_graph
-rw-r--r-- 1 root root 141585 Apr 14 2020 i2b2_assertion_sample.csv
drwxr-xr-x 2 root root 4096 Nov 1 14:00 logs
-rw-r--r-- 1 root root 4763125 Nov 1 11:57 petfinder-mini.csv
-rw-r--r-- 1 root root 4763125 Nov 1 12:50 petfinder-mini.csv.1
-rw-r--r-- 1 root root 4763125 Nov 1 13:05 petfinder-mini.csv.2
-rw-r--r-- 1 root root 4763125 Nov 1 13:10 petfinder-mini.csv.3
-rw-r--r-- 1 root root 9363435 Apr 9 2020 pubmed_sample_text_small.csv
drwxr-xr-x 2 root root 4096 Nov 1 13:53 tf_graphs
drwxr-xr-x 2 root root 4096 Nov 1 13:53 training_logs

In [0]:

assertion_df = spark.read.option("header", True).option("inferSchema", "True").csv("i2b2_assertion_sample.csv")


In [0]:
assertion_df.show(3, truncate=100)

+-----------------------------------------------------------------------------+--------------------+-------+-----+---+
 text| target| label|start|end|
+-----------------------------------------------------------------------------+--------------------+-------+-----+---+
Due to dye load required to assess the coronary disease were not visualized .|the coronary disease|present| 7| 9|
 She has no history of liver disease , hepatitis .| liver disease| absent| 5| 6|
 1. Undesired fertility .| undesired fertility|present| 1| 2|
+-----------------------------------------------------------------------------+--------------------+-------+-----+---+
only showing top 3 rows

In [0]:
(training_data, test_data) = assertion_df.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 796
Test Dataset Count: 204

In [0]:
training_data.groupBy('label').count().orderBy('count', ascending=False).show(truncate=False)


+----------------------------+-----+
label |count|
+----------------------------+-----+
present |529 |
absent |184 |
hypothetical |37 |
possible |36 |
associated_with_someone_else|6 |
conditional |4 |
+----------------------------+-----+

In [0]:

document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

chunk = Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("chunk")\
    .setChunkCol("target")\
    .setStartCol("start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(True)

token = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["document", "token"])\
      .setOutputCol("embeddings")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
!mkdir training_logs
!mkdir tf_graphs
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/nerdl_graph/blstm_34_32_30_200_6.pb -P /databricks/driver/tf_graphs

In [0]:

assertionStatus = AssertionDLApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(128)\
    .setDropout(0.1)\
    .setLearningRate(0.001)\
    .setEpochs(10)\
    .setValidationSplit(0.2)\
    .setStartCol("start")\
    .setEndCol("end")\
    .setMaxSentLen(250)\
  .setClasses(6)\
.setEnableOutputLogs(True)\
.setOutputLogsPath('training_logs/')\
.setGraphFolder('file:/databricks/driver/tf_graphs')

# set this path to graphs if you have different nuber of classes other than 6. (use 2.1 Asertion Model Graph generation notebook)

In [0]:

from sparknlp_jsl.training import tf_graph

import sys

sys.path.append('/databricks/driver/tf_graph') 

tf_graph.print_model_params("assertion_dl")

assertion_dl parameters.
Parameter Required Default value Description
max_seq_len no 250 Maximum sequence length.
feat_size no 200 Feature size.
n_classes yes - Number of classes.
device no /cpu:0 Device for training.
n_hidden no 34 Number of hidden units.

In [0]:
max_seq_len = 250
feat_size = 200
n_classes = 7


tf_graph.build("assertion_dl",build_params={"n_classes": n_classes, 'max_seq_len':max_seq_len, 'n_classes':n_classes}, model_location="dbfs:/databricks/driver/tf_graphs", model_filename="auto")

assertion_dl graph exported to dbfs:/databricks/driver/tf_graphs/blstm_250_200_34_7.pb

In [0]:
'''
assertionStatus = AssertionLogRegApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setMaxIter(100) # default: 26
'''

In [0]:
clinical_assertion_pipeline = Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings,
    assertionStatus])

In [0]:
#%%time

assertion_model = clinical_assertion_pipeline.fit(training_data)

In [0]:
%sh cd training_logs && ls -lt

total 8
-rw-r--r-- 1 root root 6714 Nov 1 14:08 AssertionDLApproach_037b304ea9cc.log

In [0]:
%sh tail -n 26 training_logs/AssertionDLApproach_037b304ea9cc.log

present 96 46 11 0.6760563 0.89719623 0.7710843
conditional 0 0 1 0.0 0.0 0.0
associated_with_someone_else 0 0 3 0.0 0.0 0.0
hypothetical 0 0 8 0.0 0.0 0.0
possible 0 0 5 0.0 0.0 0.0
tp: 102 fp: 57 fn: 57 labels: 6
Macro-average prec: 0.1714996, rec: 0.17810415, f1: 0.1747395
Micro-average prec: 0.6415094, rec: 0.6415094, f1: 0.6415094


Epoch: 9 started, learning rate: 6.302493E-4, dataset size: 637
Done, 4.794174729 loss: 4.8547945, batches: 5
Quality on validation dataset (20.0%), validation examples = 159
time to finish evaluation: 1.06s
label tp fp fn prec rec f1
absent 5 11 30 0.3125 0.14285715 0.19607843
present 96 47 11 0.67132866 0.89719623 0.76800007
conditional 0 0 1 0.0 0.0 0.0
associated_with_someone_else 0 0 3 0.0 0.0 0.0
hypothetical 0 0 8 0.0 0.0 0.0
possible 0 0 5 0.0 0.0 0.0
tp: 101 fp: 58 fn: 58 labels: 6
Macro-average prec: 0.16397144, rec: 0.17334223, f1: 0.16852668
Micro-average prec: 0.6352201, rec: 0.6352201, f1: 0.6352201

In [0]:
preds = assertion_model.transform(test_data).select('label','assertion.result')

preds.show()

+------------+---------+
 label| result|
+------------+---------+
 present|[present]|
 possible|[present]|
 present|[present]|
 present|[present]|
 present| [absent]|
 present|[present]|
 present| [absent]|
 present|[present]|
 present| [absent]|
 present|[present]|
 present|[present]|
hypothetical|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present| [absent]|
 present|[present]|
 present|[present]|
hypothetical|[present]|
 present| [absent]|
+------------+---------+
only showing top 20 rows

In [0]:
preds_df = preds.toPandas()


In [0]:
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])


In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['label']))

/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
 'recall', 'true', average, warn_for)
 precision recall f1-score support

 absent 0.25 0.41 0.31 27
associated_with_someone_else 0.00 0.00 0.00 0
 conditional 0.00 0.00 0.00 0
 hypothetical 0.00 0.00 0.00 0
 possible 0.00 0.00 0.00 0
 present 0.88 0.67 0.76 177

 micro avg 0.64 0.64 0.64 204
 macro avg 0.19 0.18 0.18 204
 weighted avg 0.80 0.64 0.70 204

In [0]:
assertion_model.stages[-1].write().overwrite().save('/databricks/driver/models/assertion_model_0804')